# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [4]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_sentence = ([vocab for vocab in sentence.split(' ') if len(vocab) > 0] for sentence in source_text.split('\n'))
    source_id_text = [[source_vocab_to_int[vocab] for vocab in sentence] for sentence in source_sentence]
    target_sentence = ([vocab for vocab in sentence.split(' ') if len(vocab) > 0] + ['<EOS>'] for sentence in target_text.split('\n'))
    target_id_text = [[target_vocab_to_int[vocab] for vocab in sentence] for sentence in target_sentence]
    
    return (source_id_text, target_id_text)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [8]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_seq_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_seq_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, [None], name='source_sequence_length')
   
    return inputs, targets, learning_rate, keep_prob, target_seq_length, max_target_len, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [9]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    target_data_removed = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    go_id = tf.fill([batch_size, 1], target_vocab_to_int['<GO>'])
    decoder_input = tf.concat([go_id, target_data_removed], 1)
    
    return decoder_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [10]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    def build_lstm_cell(rnn_size, keep_prob):
        cell = tf.contrib.rnn.LSTMCell(num_units=rnn_size,initializer=tf.random_uniform_initializer(-0.1, 0.1))
        dropout_cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
        return dropout_cell
    
    embed = tf.contrib.layers.embed_sequence(ids=rnn_inputs, vocab_size=source_vocab_size, embed_dim=encoding_embedding_size)
    cells = [build_lstm_cell(rnn_size, keep_prob) for _ in range(num_layers)]
    multicell = tf.contrib.rnn.MultiRNNCell(cells)
    (RNN_output, RNN_state) = tf.nn.dynamic_rnn(cell=multicell, inputs=embed, sequence_length=source_sequence_length, dtype=tf.float32)
        
    return RNN_output, RNN_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, sequence_length=target_sequence_length)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(cell=tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob=keep_prob), 
                                                    helper=training_helper, initial_state=encoder_state, output_layer=output_layer)
    (basic_decoder_outputs, _) =  tf.contrib.seq2seq.dynamic_decode(decoder=basic_decoder, maximum_iterations=max_summary_length)
    
    return basic_decoder_outputs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_tokens = tf.tile(input=tf.constant([start_of_sequence_id], dtype=tf.int32), multiples=[batch_size], 
                           name='start_tokens')
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding=dec_embeddings, start_tokens=start_tokens, 
                                                                end_token=end_of_sequence_id)
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell, helper=inference_helper, 
                                                        initial_state=encoder_state, output_layer=output_layer)
    (inference_decoder_output, _) = tf.contrib.seq2seq.dynamic_decode(decoder=inference_decoder, impute_finished=True, 
                                                                      maximum_iterations=max_target_sequence_length)
    
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [13]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    def build_lstm_cell(rnn_size):
        cell = tf.contrib.rnn.LSTMCell(num_units=rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1))
        return cell
    
    embeddings = tf.Variable(initial_value=tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    embed_input = tf.nn.embedding_lookup(params=embeddings, ids=dec_input)
    multicell = tf.contrib.rnn.MultiRNNCell([build_lstm_cell(rnn_size) for _ in range(num_layers)])
    output_layer = Dense(units=target_vocab_size, kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))  
    with tf.variable_scope('decode'):
        training_basic_decoder_output = decoding_layer_train(
            encoder_state,
            multicell,
            embed_input,
            target_sequence_length,
            max_target_sequence_length,
            output_layer, keep_prob)
    with tf.variable_scope('decode', reuse=True):
        inference_basic_decoder_output = decoding_layer_infer(
            encoder_state,
            multicell,
            embeddings,
            target_vocab_to_int['<GO>'],
            target_vocab_to_int['<EOS>'],
            max_target_sequence_length,
            target_vocab_size,
            output_layer,
            batch_size, keep_prob)
        
    return training_basic_decoder_output, inference_basic_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    (encoded_output, encoded_state) = encoding_layer(
        input_data,
        rnn_size,
        num_layers,
        keep_prob,
        source_sequence_length,
        source_vocab_size,
        enc_embedding_size)
    decoded_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    (training_decoder_output, inference_decoder_output) = decoding_layer(
        decoded_input,
        encoded_state,
        target_sequence_length,
        max_target_sentence_length,
        rnn_size,
        num_layers,
        target_vocab_to_int,
        target_vocab_size,
        batch_size,
        keep_prob,
        dec_embedding_size)
    
    return (training_decoder_output, inference_decoder_output)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [15]:
# Number of Epochs
epochs = 8
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 256
decoding_embedding_size = 256
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.6
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/1077 - Train Accuracy: 0.2215, Validation Accuracy: 0.3050, Loss: 5.2525
Epoch   0 Batch    2/1077 - Train Accuracy: 0.2064, Validation Accuracy: 0.3050, Loss: 4.9170
Epoch   0 Batch    3/1077 - Train Accuracy: 0.2492, Validation Accuracy: 0.3182, Loss: 4.5543
Epoch   0 Batch    4/1077 - Train Accuracy: 0.2504, Validation Accuracy: 0.3285, Loss: 4.4034
Epoch   0 Batch    5/1077 - Train Accuracy: 0.2941, Validation Accuracy: 0.3430, Loss: 4.1077
Epoch   0 Batch    6/1077 - Train Accuracy: 0.2887, Validation Accuracy: 0.3469, Loss: 3.9656
Epoch   0 Batch    7/1077 - Train Accuracy: 0.3004, Validation Accuracy: 0.3746, Loss: 3.9223
Epoch   0 Batch    8/1077 - Train Accuracy: 0.3117, Validation Accuracy: 0.3739, Loss: 3.7126
Epoch   0 Batch    9/1077 - Train Accuracy: 0.3234, Validation Accuracy: 0.3832, Loss: 3.5754
Epoch   0 Batch   10/1077 - Train Accuracy: 0.2895, Validation Accuracy: 0.3846, Loss: 3.6529
Epoch   0 Batch   11/1077 - Train Accuracy: 0.3497, Validati

Epoch   0 Batch   89/1077 - Train Accuracy: 0.4473, Validation Accuracy: 0.4940, Loss: 1.6543
Epoch   0 Batch   90/1077 - Train Accuracy: 0.4336, Validation Accuracy: 0.4961, Loss: 1.6908
Epoch   0 Batch   91/1077 - Train Accuracy: 0.4948, Validation Accuracy: 0.4840, Loss: 1.5038
Epoch   0 Batch   92/1077 - Train Accuracy: 0.4528, Validation Accuracy: 0.4950, Loss: 1.6031
Epoch   0 Batch   93/1077 - Train Accuracy: 0.4320, Validation Accuracy: 0.4961, Loss: 1.6376
Epoch   0 Batch   94/1077 - Train Accuracy: 0.4406, Validation Accuracy: 0.4854, Loss: 1.5609
Epoch   0 Batch   95/1077 - Train Accuracy: 0.4650, Validation Accuracy: 0.4940, Loss: 1.5816
Epoch   0 Batch   96/1077 - Train Accuracy: 0.4359, Validation Accuracy: 0.4744, Loss: 1.5901
Epoch   0 Batch   97/1077 - Train Accuracy: 0.4777, Validation Accuracy: 0.4933, Loss: 1.5686
Epoch   0 Batch   98/1077 - Train Accuracy: 0.5074, Validation Accuracy: 0.4975, Loss: 1.4989
Epoch   0 Batch   99/1077 - Train Accuracy: 0.4227, Validati

Epoch   0 Batch  177/1077 - Train Accuracy: 0.4889, Validation Accuracy: 0.5487, Loss: 1.1111
Epoch   0 Batch  178/1077 - Train Accuracy: 0.5348, Validation Accuracy: 0.5614, Loss: 1.0342
Epoch   0 Batch  179/1077 - Train Accuracy: 0.5317, Validation Accuracy: 0.5685, Loss: 1.0822
Epoch   0 Batch  180/1077 - Train Accuracy: 0.5262, Validation Accuracy: 0.5682, Loss: 1.0314
Epoch   0 Batch  181/1077 - Train Accuracy: 0.5410, Validation Accuracy: 0.5536, Loss: 1.0611
Epoch   0 Batch  182/1077 - Train Accuracy: 0.5290, Validation Accuracy: 0.5444, Loss: 1.0028
Epoch   0 Batch  183/1077 - Train Accuracy: 0.4945, Validation Accuracy: 0.5391, Loss: 1.0431
Epoch   0 Batch  184/1077 - Train Accuracy: 0.5246, Validation Accuracy: 0.5273, Loss: 1.0024
Epoch   0 Batch  185/1077 - Train Accuracy: 0.5055, Validation Accuracy: 0.5359, Loss: 1.0206
Epoch   0 Batch  186/1077 - Train Accuracy: 0.5226, Validation Accuracy: 0.5465, Loss: 1.0580
Epoch   0 Batch  187/1077 - Train Accuracy: 0.5387, Validati

Epoch   0 Batch  265/1077 - Train Accuracy: 0.5695, Validation Accuracy: 0.5941, Loss: 0.7832
Epoch   0 Batch  266/1077 - Train Accuracy: 0.5971, Validation Accuracy: 0.5898, Loss: 0.7332
Epoch   0 Batch  267/1077 - Train Accuracy: 0.5650, Validation Accuracy: 0.5803, Loss: 0.7159
Epoch   0 Batch  268/1077 - Train Accuracy: 0.5844, Validation Accuracy: 0.6051, Loss: 0.7637
Epoch   0 Batch  269/1077 - Train Accuracy: 0.5485, Validation Accuracy: 0.5930, Loss: 0.8316
Epoch   0 Batch  270/1077 - Train Accuracy: 0.5703, Validation Accuracy: 0.5920, Loss: 0.7817
Epoch   0 Batch  271/1077 - Train Accuracy: 0.6012, Validation Accuracy: 0.5852, Loss: 0.7557
Epoch   0 Batch  272/1077 - Train Accuracy: 0.6231, Validation Accuracy: 0.5952, Loss: 0.7754
Epoch   0 Batch  273/1077 - Train Accuracy: 0.5852, Validation Accuracy: 0.5938, Loss: 0.7392
Epoch   0 Batch  274/1077 - Train Accuracy: 0.6105, Validation Accuracy: 0.6101, Loss: 0.7343
Epoch   0 Batch  275/1077 - Train Accuracy: 0.5878, Validati

Epoch   0 Batch  353/1077 - Train Accuracy: 0.5876, Validation Accuracy: 0.6019, Loss: 0.6965
Epoch   0 Batch  354/1077 - Train Accuracy: 0.6363, Validation Accuracy: 0.6069, Loss: 0.6620
Epoch   0 Batch  355/1077 - Train Accuracy: 0.6060, Validation Accuracy: 0.6303, Loss: 0.6259
Epoch   0 Batch  356/1077 - Train Accuracy: 0.6164, Validation Accuracy: 0.6151, Loss: 0.6346
Epoch   0 Batch  357/1077 - Train Accuracy: 0.6410, Validation Accuracy: 0.6225, Loss: 0.6116
Epoch   0 Batch  358/1077 - Train Accuracy: 0.6164, Validation Accuracy: 0.6261, Loss: 0.6646
Epoch   0 Batch  359/1077 - Train Accuracy: 0.6387, Validation Accuracy: 0.6183, Loss: 0.6353
Epoch   0 Batch  360/1077 - Train Accuracy: 0.6336, Validation Accuracy: 0.6154, Loss: 0.6287
Epoch   0 Batch  361/1077 - Train Accuracy: 0.6628, Validation Accuracy: 0.6300, Loss: 0.6586
Epoch   0 Batch  362/1077 - Train Accuracy: 0.6417, Validation Accuracy: 0.6300, Loss: 0.6231
Epoch   0 Batch  363/1077 - Train Accuracy: 0.5973, Validati

...

Epoch   7 Batch  660/1077 - Train Accuracy: 0.9961, Validation Accuracy: 0.9712, Loss: 0.0101
Epoch   7 Batch  661/1077 - Train Accuracy: 0.9933, Validation Accuracy: 0.9712, Loss: 0.0076
Epoch   7 Batch  662/1077 - Train Accuracy: 0.9766, Validation Accuracy: 0.9712, Loss: 0.0114
Epoch   7 Batch  663/1077 - Train Accuracy: 0.9902, Validation Accuracy: 0.9712, Loss: 0.0081
Epoch   7 Batch  664/1077 - Train Accuracy: 0.9887, Validation Accuracy: 0.9716, Loss: 0.0069
Epoch   7 Batch  665/1077 - Train Accuracy: 0.9953, Validation Accuracy: 0.9716, Loss: 0.0043
Epoch   7 Batch  666/1077 - Train Accuracy: 0.9889, Validation Accuracy: 0.9712, Loss: 0.0123
Epoch   7 Batch  667/1077 - Train Accuracy: 0.9996, Validation Accuracy: 0.9716, Loss: 0.0056
Epoch   7 Batch  668/1077 - Train Accuracy: 0.9948, Validation Accuracy: 0.9670, Loss: 0.0072
Epoch   7 Batch  669/1077 - Train Accuracy: 0.9914, Validation Accuracy: 0.9670, Loss: 0.0082
Epoch   7 Batch  670/1077 - Train Accuracy: 0.9858, Validati

Epoch   7 Batch  748/1077 - Train Accuracy: 0.9914, Validation Accuracy: 0.9684, Loss: 0.0072
Epoch   7 Batch  749/1077 - Train Accuracy: 0.9973, Validation Accuracy: 0.9684, Loss: 0.0051
Epoch   7 Batch  750/1077 - Train Accuracy: 0.9805, Validation Accuracy: 0.9684, Loss: 0.0086
Epoch   7 Batch  751/1077 - Train Accuracy: 0.9961, Validation Accuracy: 0.9751, Loss: 0.0060
Epoch   7 Batch  752/1077 - Train Accuracy: 0.9918, Validation Accuracy: 0.9759, Loss: 0.0072
Epoch   7 Batch  753/1077 - Train Accuracy: 0.9910, Validation Accuracy: 0.9737, Loss: 0.0078
Epoch   7 Batch  754/1077 - Train Accuracy: 0.9957, Validation Accuracy: 0.9741, Loss: 0.0052
Epoch   7 Batch  755/1077 - Train Accuracy: 0.9801, Validation Accuracy: 0.9691, Loss: 0.0099
Epoch   7 Batch  756/1077 - Train Accuracy: 0.9875, Validation Accuracy: 0.9691, Loss: 0.0065
Epoch   7 Batch  757/1077 - Train Accuracy: 0.9996, Validation Accuracy: 0.9691, Loss: 0.0045
Epoch   7 Batch  758/1077 - Train Accuracy: 0.9944, Validati

Epoch   7 Batch  836/1077 - Train Accuracy: 0.9918, Validation Accuracy: 0.9677, Loss: 0.0061
Epoch   7 Batch  837/1077 - Train Accuracy: 0.9824, Validation Accuracy: 0.9680, Loss: 0.0085
Epoch   7 Batch  838/1077 - Train Accuracy: 0.9910, Validation Accuracy: 0.9702, Loss: 0.0083
Epoch   7 Batch  839/1077 - Train Accuracy: 1.0000, Validation Accuracy: 0.9702, Loss: 0.0040
Epoch   7 Batch  840/1077 - Train Accuracy: 0.9879, Validation Accuracy: 0.9702, Loss: 0.0071
Epoch   7 Batch  841/1077 - Train Accuracy: 0.9863, Validation Accuracy: 0.9755, Loss: 0.0086
Epoch   7 Batch  842/1077 - Train Accuracy: 0.9863, Validation Accuracy: 0.9755, Loss: 0.0052
Epoch   7 Batch  843/1077 - Train Accuracy: 0.9907, Validation Accuracy: 0.9705, Loss: 0.0049
Epoch   7 Batch  844/1077 - Train Accuracy: 0.9885, Validation Accuracy: 0.9659, Loss: 0.0050
Epoch   7 Batch  845/1077 - Train Accuracy: 0.9930, Validation Accuracy: 0.9659, Loss: 0.0049
Epoch   7 Batch  846/1077 - Train Accuracy: 0.9879, Validati

Epoch   7 Batch  924/1077 - Train Accuracy: 0.9827, Validation Accuracy: 0.9673, Loss: 0.0070
Epoch   7 Batch  925/1077 - Train Accuracy: 0.9881, Validation Accuracy: 0.9648, Loss: 0.0064
Epoch   7 Batch  926/1077 - Train Accuracy: 0.9840, Validation Accuracy: 0.9624, Loss: 0.0096
Epoch   7 Batch  927/1077 - Train Accuracy: 0.9941, Validation Accuracy: 0.9673, Loss: 0.0112
Epoch   7 Batch  928/1077 - Train Accuracy: 0.9914, Validation Accuracy: 0.9673, Loss: 0.0073
Epoch   7 Batch  929/1077 - Train Accuracy: 0.9863, Validation Accuracy: 0.9766, Loss: 0.0060
Epoch   7 Batch  930/1077 - Train Accuracy: 0.9988, Validation Accuracy: 0.9766, Loss: 0.0062
Epoch   7 Batch  931/1077 - Train Accuracy: 0.9902, Validation Accuracy: 0.9766, Loss: 0.0057
Epoch   7 Batch  932/1077 - Train Accuracy: 0.9941, Validation Accuracy: 0.9766, Loss: 0.0060
Epoch   7 Batch  933/1077 - Train Accuracy: 0.9938, Validation Accuracy: 0.9766, Loss: 0.0074
Epoch   7 Batch  934/1077 - Train Accuracy: 0.9891, Validati

Epoch   7 Batch 1012/1077 - Train Accuracy: 0.9855, Validation Accuracy: 0.9705, Loss: 0.0058
Epoch   7 Batch 1013/1077 - Train Accuracy: 0.9870, Validation Accuracy: 0.9759, Loss: 0.0064
Epoch   7 Batch 1014/1077 - Train Accuracy: 0.9836, Validation Accuracy: 0.9759, Loss: 0.0057
Epoch   7 Batch 1015/1077 - Train Accuracy: 0.9930, Validation Accuracy: 0.9659, Loss: 0.0077
Epoch   7 Batch 1016/1077 - Train Accuracy: 0.9881, Validation Accuracy: 0.9709, Loss: 0.0072
Epoch   7 Batch 1017/1077 - Train Accuracy: 0.9885, Validation Accuracy: 0.9709, Loss: 0.0048
Epoch   7 Batch 1018/1077 - Train Accuracy: 0.9866, Validation Accuracy: 0.9759, Loss: 0.0053
Epoch   7 Batch 1019/1077 - Train Accuracy: 0.9856, Validation Accuracy: 0.9759, Loss: 0.0115
Epoch   7 Batch 1020/1077 - Train Accuracy: 0.9984, Validation Accuracy: 0.9741, Loss: 0.0064
Epoch   7 Batch 1021/1077 - Train Accuracy: 0.9967, Validation Accuracy: 0.9741, Loss: 0.0048
Epoch   7 Batch 1022/1077 - Train Accuracy: 0.9918, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [21]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    unknow_id = vocab_to_int['<UNK>']
    vocabs = (word.lower() for word in sentence.split(' ') if len(word) > 0)
    ids = [vocab_to_int.get(vocab, unknow_id) for vocab in vocabs]
    return ids

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [22]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [125, 122, 208, 116, 100, 113, 99]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [117, 254, 290, 282, 202, 138, 212, 136, 1]
  French Words: il a vu un vieux camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.